# Differential Evolution (DE) Algorithm

In [28]:
#------------------------------------------------------------------------------+
#   CHIA E TUNGOM
#   Simple Differential Evolution (DE) Algorithm in Python
#   Feb, 2020
#------------------------------------------------------------------------------+

In [29]:
# Import dependensies

import random
import math

# Initialize Population 

In [30]:
# 1. Start by defining the search bound then generate vectors with given predefined dimensions 

dim = 3                                                 # dimension of vector
bound = (-10, 10)                                     # (lower, upper) bound for one dimensional vector
bounds = [bound for i in range(dim)]                # for a 3 dimensionsal vector

print(bounds)

# 2. generate a population of random vectors (X) within the bound each with dimension D (dim)

pop = 5
X = [[random.uniform(bounds[i][0], bounds[i][1]) for i in range(dim)] for i in range(pop)]
print(X)

[(-10, 10), (-10, 10), (-10, 10)]
[[1.123333685412609, -8.672887343308249, -5.341757321416187], [-7.7460425236966435, 4.53536414185006, -1.046214806465116], [-4.66401793920838, -9.590596022493521, -3.8180108795864403], [-4.309358043632336, -1.1414814506136786, -0.515760893834516], [-3.762620284230964, 4.775940649245129, -5.988129840983129]]


# Mutation Operation 

In [31]:
# 3. generate a mutant vector (V) for each target vector 
#   V(i,g+1) = X(r1, g) + F *(X(r2, g)-X(r3, g))       r1 != r2 != r3   F = mutation rate [0,2]
#                                                      DE best 1 scheme X(r1, g) = X(best, g)      g= generation

index = [i for i in range(dim)]

def selectI(I, Vector, n=3):   
    """" I: the vector which cannot be selected
         Vector: List of vectors 
         n: number of indices to be selected    """
    si = []
    V = Vector.copy()        # so not to alter Vector
    V.remove(I)
    for i in range (n):
        a = random.choice(V)
        si.append(a)
        V.remove(a)
    return si 

Sel = selectI(X[0], X, 3)
print(Sel)

F = random.uniform(0,2)
print(F)

def MutateX(a,b,c,F=1.3):
    
    d = []
    e = []
    
    for i in range(len(a)):
        d.append( F*(b[i]-c[i]) )
        
    for i in range (len(a)):
        e.append( a[i] + d[i] )
        
    return e

MutateX(Sel[0], Sel[1], Sel[2], random.uniform(0,2))

V=[]
for i in range(pop):
    
    Sel = selectI(X[i], X, 3)
    
    V.append( MutateX(Sel[0], Sel[1], Sel[2], random.uniform(0,2)) )
    
V   

[[-4.66401793920838, -9.590596022493521, -3.8180108795864403], [-4.309358043632336, -1.1414814506136786, -0.515760893834516], [-7.7460425236966435, 4.53536414185006, -1.046214806465116]]
1.5003539470104397


[[-4.311678335093483, -5.777173363835716, -7.344623547641642],
 [-4.349755873192657, -1.7853447851056443, -0.4185029351033936],
 [11.987366099099415, -18.678897225958607, -13.616032246341875],
 [6.861744968829574, -8.326319276563186, -12.460947628384508],
 [-13.376063363520611, 3.383358875964351, 0.4013344982635889]]

# CrossOver Operation 

In [32]:
# CrossOver Operator

# 4. generate a trial vectors (U) using target and mutant vector (Binomial variant)

# select V(i, g+1)^j   if rand(j) <= CR or j = a(j)    CR = crossOver probability, a(j) = selected dimension of U(i, g+1)
# else select x(i,g)

#CR = 0.5
#randj = [random.random() for i in range(dim)]
#indexs = [i for i in range(dim)]

def BinomialU(X, V, dim=dim, pop=pop, CR = 0.5):
    """ X: vectors
        V: Mutant vectors
        returns U: trial vector"""
    U = []
    indexs = [i for i in range(dim)]
    CR = 0.5
    
    for i in range(pop):
        randj = [random.random() for i in range(dim)]
        rj = random.choice(indexs)
        U.append([])
        
        for j in range(dim):
            if randj[j] <= CR or j == rj:
                U[i].append(V[i][j])
            else:
                U[i].append(X[i][j])
            
    return U


U = BinomialU(X,V)     
U

[[-4.311678335093483, -8.672887343308249, -7.344623547641642],
 [-7.7460425236966435, -1.7853447851056443, -1.046214806465116],
 [11.987366099099415, -9.590596022493521, -13.616032246341875],
 [6.861744968829574, -1.1414814506136786, -12.460947628384508],
 [-13.376063363520611, 3.383358875964351, -5.988129840983129]]

# Selection Operation 

In [33]:
# compare fitness of Ui and Xi.the better one survives 

# 5. Define objective function 

def sphere(x):
    ans = 0
    for i in range(len(x)):
        ans += x[i]**2
        
    return ans

# 6. Use objective function to select next generation

def Selection(X, U, pop=pop):
    """" U: trial vector
         X: original Vector
         returns a New X with either U[i] or X[i] at position i """
    NewX = []
    for i in range(pop):
        if sphere(U[i]) < sphere(X[i]):
            
            NewX.append(U[i])
        else:
            NewX.append(X[i])
        
    return NewX

Selection(X,U)

[[1.123333685412609, -8.672887343308249, -5.341757321416187],
 [-7.7460425236966435, -1.7853447851056443, -1.046214806465116],
 [-4.66401793920838, -9.590596022493521, -3.8180108795864403],
 [-4.309358043632336, -1.1414814506136786, -0.515760893834516],
 [-3.762620284230964, 4.775940649245129, -5.988129840983129]]

In [34]:
min ([sphere(i) for i in X])

20.1395559499228

# DE (Putting it all together) 

In [48]:
# Write a DE Loop

# input: dim, pop, bounds, function

def DE(dim, bound, function, pop = 10, runs = 20):
    
    F = 0.5
    bounds = [bound for i in range(dim)]
    X = [[random.uniform(bounds[i][0], bounds[i][1]) for i in range(dim)] for i in range(pop)]
    
    EvolutionMatrix = []
    EvolutionMatrix.append(X)
    
    for i in range(runs):
        
        V = []
        S = []
        #print("HERE NOW")
        
        for j in range(pop):
            S = selectI(X[j], X, 3)
            V.append(MutateX(S[0], S[1], S[2], F))
            
        U = BinomialU(X, V, dim, pop)
        X = Selection(X, U, pop)
        f = [function(i) for i in X]
        best = min(f)
        I = f.index(best)     # find index of best position 
        
        EvolutionMatrix.append(X)
        
        #print("ITERATION " , i , "BEST:  ", min([function(i) for i in X]))
        
    print(X[I]) 
    
    return min([function(i) for i in X]), EvolutionMatrix 

# Example: Find minimum of a 5D sphere 

In [50]:

dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, sphere, pop = 20, runs= 1000)

print(fx)

[7.359243309193974e-44, 2.325744558796283e-43, 7.404263148623624e-44, -2.8446795068641423e-43, 1.5530784682848962e-43]
1.7003157726773314e-85


# Simulate 2D Evolution 

In [51]:
dimensions = 2
bound = (-10,10)

fx, Swarm = DE(dimensions, bound, sphere, pop = 10, runs= 100)

[-4.137016180984583e-11, 2.2138839063817058e-11]


In [52]:
import pandas as pd

#d={"Particle": 1, "Evolution":None, "Position":None, "Velocity":None, "best_Position":None, "Error":None,  "best_Error":None }
#df = pd.DataFrame(data = d, index = "Particle")

# make a dataframe from original swarm data structure
SF = pd.DataFrame(Swarm)

SF.head()

,0,1,2,3,4,5,6,7,8,9
0,"[-7.7736722477822955, -3.027641368060321]","[1.419999582725195, -6.321539856235033]","[2.1830793417919097, -7.687879048081621]","[3.0941830052089454, 0.5235317648432929]","[0.5457215916529599, -8.323136929686958]","[-3.3470510948975774, -5.782859309450408]","[-1.562070328863518, -6.495320443064649]","[-3.076453926331329, -9.912334646262952]","[-9.539681405194091, 0.05163804606282696]","[2.318806488017948, -4.744467257018156]"
1,"[4.148078965467184, -0.3903764784678616]","[1.419999582725195, -4.097267791524158]","[2.1830793417919097, -2.013099622343445]","[3.0941830052089454, 0.5235317648432929]","[1.5616146892840428, -6.452974358617308]","[-3.3470510948975774, -5.782859309450408]","[-4.509244363880825, -3.923487204694541]","[4.525800542184584, -8.055379393737198]","[-9.539681405194091, 0.05163804606282696]","[2.318806488017948, -4.744467257018156]"
2,"[4.148078965467184, -0.3903764784678616]","[1.419999582725195, -4.097267791524158]","[2.1830793417919097, -2.013099622343445]","[3.0941830052089454, 0.5235317648432929]","[-0.06471885079343798, -6.452974358617308]","[-3.3470510948975774, -5.782859309450408]","[-4.509244363880825, -3.923487204694541]","[-5.036192344009944, -3.466533083038964]","[-9.539681405194091, 0.05163804606282696]","[2.318806488017948, -4.744467257018156]"
3,"[4.148078965467184, -0.3903764784678616]","[1.419999582725195, -4.097267791524158]","[1.0323113241296964, -2.013099622343445]","[3.0941830052089454, 0.5235317648432929]","[-0.06471885079343798, -6.452974358617308]","[-3.3470510948975774, -5.782859309450408]","[-3.456741416008752, -3.923487204694541]","[0.9705961300788184, -3.466533083038964]","[-5.569313851441271, -4.518115732489024]","[2.318806488017948, -4.744467257018156]"
4,"[-4.099988997952877, -0.3903764784678616]","[1.419999582725195, -4.097267791524158]","[1.0323113241296964, -2.013099622343445]","[3.0941830052089454, 0.5235317648432929]","[0.6558125058419719, 0.6104220582581013]","[1.450857179750634, -5.782859309450408]","[1.3957814651432683, -3.923487204694541]","[0.9705961300788184, -3.466533083038964]","[-5.569313851441271, -4.518115732489024]","[-0.6705699919732513, -4.744467257018156]"


In [53]:
# Write a Loop to populate Dataframe of Swarm

DF1 = pd.DataFrame(columns=["Particle","Evolution","Position"])

for i in range(len(Swarm)):
    for j in range(len(Swarm[0])):
        
        DF1 = DF1.append({"Particle": j, "Evolution": i, "Position":Swarm[i][j]}, ignore_index=True)
        
        
DF1.head()

,Particle,Evolution,Position
0,0.0,0.0,"[-7.7736722477822955, -3.027641368060321]"
1,1.0,0.0,"[1.419999582725195, -6.321539856235033]"
2,2.0,0.0,"[2.1830793417919097, -7.687879048081621]"
3,3.0,0.0,"[3.0941830052089454, 0.5235317648432929]"
4,4.0,0.0,"[0.5457215916529599, -8.323136929686958]"


In [54]:
DF1[["PositionX", "PositionY"]] = pd.DataFrame(DF1.Position.values.tolist(), index = DF1.index)
DF1["size"] = 5
DF1.head()

,Particle,Evolution,Position,PositionX,PositionY,size
0,0.0,0.0,"[-7.7736722477822955, -3.027641368060321]",-7.773672,-3.027641,5
1,1.0,0.0,"[1.419999582725195, -6.321539856235033]",1.420000,-6.321540,5
2,2.0,0.0,"[2.1830793417919097, -7.687879048081621]",2.183079,-7.687879,5
3,3.0,0.0,"[3.0941830052089454, 0.5235317648432929]",3.094183,0.523532,5
4,4.0,0.0,"[0.5457215916529599, -8.323136929686958]",0.545722,-8.323137,5


In [55]:
import plotly.express as px

px.scatter(DF1, x="PositionX", y="PositionY", animation_frame="Evolution", animation_group="Particle",
           size= "size", color="Particle", size_max=13, range_x=[-10,10], range_y=[-10,10])

# Optimize other Benchmark functions 

In [56]:
# Define the funtions 

def sphere(x):
    ans = 0
    for i in range(len(x)):
        ans += x[i]**2
        
    return ans


def wsphere(x):
    ans  = 0
    for i in range(len(x)):
        ans += i*(x[i]**2)
        
    return ans

def schewefels(x, n = 4):
    ans = 0
    for i in range(len(x)):
        ans += sphere(x) 
    return ans 

def rhellipsoid(x, n = 4):
    ans  = 0
    for i in range(len(x)):
        fx = 0
        for i in range(len(x)):
            fx += x[i]
        ans += fx**2
    return ans 

def rosenbrock(x):
    ans  = 0
    for i in range(len(x)-1):
        ans += (100 * ( x[i+1] - x[i] )**2) + (x[i]-1)**2
    return ans
        
    
def rastrigen(x):
    ans  = 0
    for i in range (len(x)):
        ans += x[i]**2 - 10*( math.cos(2*math.pi*x[i]) ) + 10
    return ans   

def griewank(x):
    ans = 0
    a = 0
    b = 1
    for i in range(len(x)):
        a += x[i]**2
        b *= math.cos( x[i] / ((i+1)**(1/2)) )
    ans = ( (1/4000)*a ) + b + 1
    return ans 

def aukley(x):
    a = 0
    b = 0
    ans = 0
    for i in range(len(x)):
        a += x[i]**2
        b += math.cos( 2*math.pi*x[i])
    ans = (-20*math.exp((-0.2)*(math.sqrt( (1/len(x)) *a)) ) )- math.exp( (1/len(x)) * b ) + 20 + math.e
    return ans


def schewefels2(x):
    ans = 0
    a = 0
    b = 1
    for i in range(len(x)):
        a += x[i]
        b *= x[i]
    ans = a + b 
    return ans 


In [57]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, wsphere, pop = 20, runs= 1000)
fx

[-4.218008790229175e-46, -1.4456595155140985e-44, 7.477827693588236e-45, 9.619572879535127e-46, 9.304311136721506e-46]


3.270678512103336e-88

In [58]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, schewefels, pop = 20, runs= 1000)
fx

[1.1776157488169092e-44, 8.606098850208256e-46, 4.214618276340418e-45, 1.2133911928748379e-44, -6.508688356701362e-45]


1.7338819229745063e-87

In [59]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, rhellipsoid, pop = 20, runs= 1000)
fx

[2.5044561408388934e-44, 2.292660782273833e-44, 1.0805509930317905e-44, -4.573611860409702e-44, -8.832821215899351e-45]


8.852535182787994e-89

In [60]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, rosenbrock, pop = 20, runs= 1000)
fx

[-3.3341186319252615e-44, -3.597080377506173e-46, 2.0111689315588964e-44, -3.804903561829609e-44, -1.935082530554098e-43]


4.0

In [61]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, rastrigen, pop = 20, runs= 1000)
fx

[5.1388897813737774e-45, -5.5689830755366244e-46, 7.143430331158547e-44, -7.992699092214782e-44, 4.041966929626008e-44]


0.0

In [27]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, griewank, pop = 20, runs= 1000)
fx

[-4.950878595299777e-44, -8.056127228442869e-45, -4.262549238282375e-45, 3.9350817273287234e-45, 3.431577592579864e-44]


2.0

In [22]:
dimensions = 5
bound = (-10,10)

fx, M = DE(dimensions, bound, aukley, pop = 20, runs= 1000)
fx

[9.200162999898052e-47, 1.9576310906573923e-45, -1.913823157766957e-45, -7.742700229513765e-46, 2.1785765610848497e-46]


4.440892098500626e-16

In [23]:
dimensions = 5
bound = (-10,10)

fx, M  = DE(dimensions, bound, schewefels2, pop = 20, runs= 1000)
fx

[-3.8174020303556047e-45, 2.3744683598633694e-44, -6.505491703489721e-45, -6.300600043971876e-44, -1.3092882116144903e-46]


-4.971513939609185e-44